# 6 Visualise labelled data and remove artifacts

Code that let us overlay each frame of video with outputs from the models. And create time series plots. 

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import ultralytics.utils as ultrautils
import utils
import display
import json
import time

## Test data

In [ ]:
videos_in = os.path.join("..","LookitLaughter.test")
demo_data = os.path.join("..","data", "demo")
temp_out = os.path.join("..","data","0_temp")
data_out = os.path.join("..","data","1_interim")
videos_out = os.path.join("..","data","2_final")

metadata_file = "_LookitLaughter.xlsx"

## Full data

In [ ]:
videos_in = os.path.join("..","..","LookitLaughter.full.videos")
temp_out = os.path.join("..","..","LookitLaughter.full.data","0_temp")
data_out = os.path.join("..","..","LookitLaughter.full.data","1_interim")
videos_out = os.path.join("..","..","LookitLaughter.full.data","2_final")

metadata_file = "_LookitLaughter.xlsx"

In [1]:
processedvideos = utils.getProcessedVideos(data_out)
processedvideos.head()

NameError: name 'utils' is not defined

## 6.1 Add annotations to all vidoes.

Generate annotated videos for all videos in the test set.

In [ ]:
forceAnnotation = False

for index, r in processedvideos.iterrows():

    videopath = os.path.join(videos_in,r["VideoID"])
    videoname = os.path.basename(r["VideoID"])
    try: 
        #let's get all the annotations for this video
        kpts = utils.getKeyPoints(processedvideos,videoname)
        facedata = utils.getFaceData(processedvideos,videoname)
        speechdata = utils.getSpeechData(processedvideos,videoname)
    except FileNotFoundError as e:
        print(f"Data error for {videoname}\n" + "Error: " + str(e))
        continue
    if forceAnnotation or pd.isnull(r["annotatedVideo"]) or not os.path.exists(r["annotatedVideo"]):
        print(f"Creating annotated video for {videoname}")
        annotatedVideo = display.createAnnotatedVideo(videopath, kpts, facedata, speechdata, temp_out, False)
        vidwithaudio = display.addSoundtoVideo(annotatedVideo, r["Audio.file"], videos_out)
        r["annotatedVideo"] = vidwithaudio
        r["annotated.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
    else:
        print(f"Already processed {r['VideoID']}")

#save the processedvideos dataframe
utils.saveProcessedVideos(processedvideos, data_out)

In [ ]:
#facedata = utils.getFaceData(processedvideos,videoname)
videodata = processedvideos[processedvideos["VideoID"] == videoname]
if videodata.shape[0] <= 0:
    raise FileNotFoundError(f"No face data file found for {videoname}")
print(f"We have a face data file for {videoname}")
facesfile = videodata["Faces.file"].values[0]
print(facesfile)
pd.read_csv(facesfile)


# 6.2  Correct person labels in all videos.

Swap parent and child if these are wrong. Ignore other people in video.

In metadata, we have 

In [ ]:


def keyPointsStdev(df, frames=[], people="all", bodypart="whole"):
    """find standard deviation of x and y values of keypoints        

    args:   df - dataframe of keypoints
            frames - list of frames to include
            people - list of people to include
            bodypart - which bodypart to use, default is "whole" for all keypoints
    returns:
            dataframe of average positions
    """

    if len(frames) == 0:
        frames = df.frame.unique()

    if people == "all":
        people = df.person.unique()

    if bodypart != "whole":
        raise NotImplementedError("Only whole body implemented for now")

    threshold = 0.5

    # create new columns for the centre of gravity
    df["std.x"] = np.nan
    df["std.y"] = np.nan

    for frame in frames:
        for person in people:
            # get the keypoints for this person in this frame
            kpts = df[(df["frame"] == frame) & (df["person"] == person)]

            if not kpts.empty:
                # get the average position of the bodypart
                if bodypart == "whole":
                    xyc = kpts.iloc[:, 8:59].to_numpy()  # just keypoints
                    xyc = xyc.reshape(-1, 3)  # reshape to n x 3 array (x,y,conf
                    avgx, avgy = stdevxys(xyc, threshold)

                df.loc[
                    (df["frame"] == frame) & (df["person"] == person), "cog.x"
                ] = avgx
                df.loc[
                    (df["frame"] == frame) & (df["person"] == person), "cog.y"
                ] = avgy

    return df